In [2]:
#importando bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## <span style="color: red;">Estudandos os dados:</span>

In [8]:
#verficando os dados
hypothesis = pd.read_csv('datasets/hypotheses_us.csv')
orders = pd.read_csv('datasets/orders_us.csv', nrows=100)
visits = pd.read_csv('datasets/visits_us.csv', nrows=100)

In [11]:
#avaliando a qualidade dos dados em orders

print(orders.info())
orders.head()

group
B    63
A    37
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   transactionId  100 non-null    int64  
 1   visitorId      100 non-null    int64  
 2   date           100 non-null    object 
 3   revenue        100 non-null    float64
 4   group          100 non-null    object 
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ KB
None


,transactionId,visitorId,date,revenue,group
0,3667963787,3312258926,2019-08-15,30.4,B
1,2804400009,3642806036,2019-08-15,15.2,B
2,2961555356,4069496402,2019-08-15,10.2,A
3,3797467345,1196621759,2019-08-15,155.1,B
4,2282983706,2322279887,2019-08-15,40.5,B


'date' sera convertido para datetime e 'group' para category.

Apesar das colunas 'transactionId' e 'visitorId' não sequirem snake_case seguiremos assim como foi feito em aula.

In [12]:
#avaliando a qualidade dos dados em visits

print(visits.info())
visits.head()

group
A    31
B    31
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    62 non-null     object
 1   group   62 non-null     object
 2   visits  62 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 1.6+ KB
None


,date,group,visits
0,2019-08-01,A,719
1,2019-08-02,A,619
2,2019-08-03,A,507
3,2019-08-04,A,717
4,2019-08-05,A,756


mesmo procedimento que em orders

In [13]:
orders = pd.read_csv('datasets/orders_us.csv', dtype={'group':'category'}, parse_dates=['date'])
visits = pd.read_csv('datasets/visits_us.csv', dtype = {'group':'category'}, parse_dates=['date'])
print(orders.info())
print(visits.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   transactionId  1197 non-null   int64         
 1   visitorId      1197 non-null   int64         
 2   date           1197 non-null   datetime64[ns]
 3   revenue        1197 non-null   float64       
 4   group          1197 non-null   category      
dtypes: category(1), datetime64[ns](1), float64(1), int64(2)
memory usage: 38.8 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    62 non-null     datetime64[ns]
 1   group   62 non-null     category      
 2   visits  62 non-null     int64         
dtypes: category(1), datetime64[ns](1), int64(1)
memory usage: 1.2 KB
None


Depois de avaliado os dados, precisamos conferir se os dados fazem sentido para um teste A/B.
Iniciamos conferindo os grupos, que devem ser iguais ou com diferenças insignificantes.

In [14]:
print(orders['group'].value_counts())
print(visits['group'].value_counts())

group
B    640
A    557
Name: count, dtype: int64
group
A    31
B    31
Name: count, dtype: int64


os grupos estao perfeitamente divididos em visits, mas nao em orders, precisamos entender se essas diferenças são significativas.
Como 'orders' se trata do numero de pedidos alguns clientes podem ter feitos mais pedidos, vamos comparar a quantidades de clientes unicos.

In [16]:
ordersA = orders.query('group == "A"')
ordersB = orders.query('group == "B"')
print(ordersA['visitorId'].nunique())
print(ordersB['visitorId'].nunique())

503
586


Os valores nao batem.
Será que os usuarios podem estar sendo contados nos dois grupos?


In [38]:
orders_grouped = orders.groupby(['visitorId','group'], observed=True)['revenue'].sum()
print(orders_grouped)

visitorId   group
5114589     B          10.8
6958315     B          25.9
8300375     A          30.5
            B         165.7
11685486    A         100.4
                      ...  
4266935830  B        1220.2
4278982564  A         385.7
4279090005  B         105.3
4281247801  B          45.6
4283872382  B          60.6
Name: revenue, Length: 1089, dtype: float64


O visitante 8300375 tem compras nos dois grupos!!
devem ter outros casos

In [61]:
#agrupando dados por visitorId e group e somando a receita para ifentificar quem tem receita em A e B
orders_grouped = orders.groupby(['visitorId','group'], observed=True, as_index=False)['revenue'].sum()

#criando uma lista de visitantes duplicados, ou seja, aqueles que tem receita em A e B
list_duplicated_id = orders_grouped['visitorId']
list_duplicated_id = list_duplicated_id[list_duplicated_id.duplicated()].reset_index(drop=True)
print(list_duplicated_id.info())


<class 'pandas.core.series.Series'>
RangeIndex: 58 entries, 0 to 57
Series name: visitorId
Non-Null Count  Dtype
--------------  -----
58 non-null     int64
dtypes: int64(1)
memory usage: 596.0 bytes
None


Foram contados 58 usuarios com esse perfil. Um valor significativo, pois se trata de 10% dos usuarios de cada grupo.

In [63]:
orders_ = orders.query('visitorId not in @list_duplicated_id')
orders_.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1016 entries, 0 to 1196
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   transactionId  1016 non-null   int64         
 1   visitorId      1016 non-null   int64         
 2   date           1016 non-null   datetime64[ns]
 3   revenue        1016 non-null   float64       
 4   group          1016 non-null   category      
dtypes: category(1), datetime64[ns](1), float64(1), int64(2)
memory usage: 40.8 KB


In [64]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   transactionId  1197 non-null   int64         
 1   visitorId      1197 non-null   int64         
 2   date           1197 non-null   datetime64[ns]
 3   revenue        1197 non-null   float64       
 4   group          1197 non-null   category      
dtypes: category(1), datetime64[ns](1), float64(1), int64(2)
memory usage: 38.8 KB


Uma parcela significativa dos dados foi comprometida

## <span style="color: red;">Priorizando hipóteses:</span>